In [1]:
from mxnet import nd
from mxnet.gluon import nn

class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')
        self.output = nn.Dense(10)
        
    def forward(self, x):
        return self.output(self.hidden(x))

In [5]:
x = nd.random.uniform(shape=(2, 20))
net = MLP()
net.initialize()
net(x)


[[-0.01377599  0.00100844 -0.00776885 -0.00708845 -0.02300985  0.07568771
   0.01597246 -0.0223236  -0.01456004 -0.04719913]
 [-0.00807849 -0.02201496  0.02650089 -0.00994154 -0.02197956  0.1080502
   0.01423418 -0.00707919 -0.03766317 -0.05399122]]
<NDArray 2x10 @cpu(0)>

In [9]:
class MySequential(nn.Block):
    def __init__(self, **kwargs):
        super(MySequential, self).__init__(**kwargs)
        
    def add(self, block):
        self._children[block.name] = block
        
    def forward(self, x):
        for block in self._children.values():
            x = block(x)
        return x

In [10]:
net = MySequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(x)


[[ 0.01368438 -0.01091824  0.05629551 -0.03863586 -0.01170888  0.05258086
  -0.01871866  0.03860081 -0.00970408  0.07207241]
 [-0.05650425  0.03088466 -0.01317807 -0.02505111 -0.01413123  0.05485576
   0.00181481  0.02240649 -0.00631516  0.04891668]]
<NDArray 2x10 @cpu(0)>

In [18]:
class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        self.rand_weight = self.params.get_constant('rand_weight', nd.random.uniform(shape=(20, 20)))
        self.dense = nn.Dense(20, activation='relu')
    def forward(self, x):
        x = self.dense(x)
        x = nd.relu(nd.dot(x, self.rand_weight.data()) + 1)
        x = self.dense(x)
        while x.norm().asscalar() > 1:
            x /= 2
        if x.norm().asscalar() < 0.8:
            x *= 10
        return x.sum()

In [19]:
net = FancyMLP()
net.initialize()
net(x)


[19.614882]
<NDArray 1 @cpu(0)>

In [20]:
class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(
            nn.Dense(64, activation='relu'),
            nn.Dense(32, activation='relu')
        )
        self.dense = nn.Dense(16, activation='relu')
    
    def forward(self, x):
        return self.dense(self.net(x))

net = nn.Sequential()
net.add(NestMLP(), nn.Dense(20), FancyMLP())

net.initialize()
net(x)


[21.755554]
<NDArray 1 @cpu(0)>